# Obtain data of AGN from SIMBAD and filtering the objects with data of Ne II, Ne III and Ne V in infrared from NED 

In this notebook we develop the code to Obtain the data of AGNs type Sy1 (Seyfert 1), Sy2 (Seyfert 2) and SyG (Seyfert Galaxies) from **[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-tap)** (SMB) and then search each object from the SMB sample in **[NED](https://ned.ipac.caltech.edu/)** to obtain the photometric data and filter all the objects from the sample that contain data of the emission lines of Ne II, Ne III and Ne V in infrared.

The part of the code to obtain the data from SIMBAD is based in Andres Ramos's work to obtain Seyfert Samples (https://github.com/aframosp/AGNView)

In [ ]:
# If this notebook will be run in Google Colab, first at all the next code must be run. If the notebook will be run in Jupyter
#then "comment" or omit this part 

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install astroquery

In [1]:
from datetime import date
import time #library to count the execution time of the search loop

import pyvo
import numpy as np
from astroquery.ned import Ned

import astropy.units as u
from astropy.table import hstack
from astropy.coordinates import SkyCoord

Using the package **[PyVO](https://pyvo.readthedocs.io/en/latest/#)** we can access to remote data and services like the **TAP Service of SIMBAD [SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-tap)** to find and retrieve the astronomical data that we need, in this case: Seyfert Galaxies. To achive that, we use a script with **[ADQL](https://www.ivoa.net/documents/latest/ADQL.html) (Astronomical Data Query Language)**, which is based on SQL.

On the script we used the next **Field Names** for VOTable ouput:
- ra: Right ascension
- dec: Declination
- coo_bibcode: Bibliographical reference for coordinates
- otype_txt: Object type
- rvz_bibcode: Bibliographical reference of radial velocity and redshift
- rvz_radvel: Stored value. Either a radial velocity, or a redshift, depending on the rvz_type field 
- rvz_redshift
- rvz_type: Stored type of velocity: 'v'=radial velocity, 'z'=redshift
- nbref: bibliographical reference

~~~~sql
SELECT main_id
	,ra
	,dec
	,coo_bibcode
	,otype_txt
	,rvz_bibcode
	,rvz_radvel
	,rvz_redshift
	,rvz_type
	,nbref
	,alltypes.otypes
FROM basic
JOIN alltypes ON oid = oidref
WHERE basic.otype = 'Sy1'
	OR basic.otype = 'Sy2'
	OR basic.otype = 'SyG'
~~~~

In [2]:
service = pyvo.dal.TAPService("http://simbad.u-strasbg.fr:80/simbad/sim-tap")

result = service.search("""
SELECT main_id
    ,ra
    ,dec
    ,coo_bibcode
    ,otype_txt
    ,rvz_bibcode
    ,rvz_radvel
    ,rvz_redshift
    ,rvz_type
    ,nbref
    ,alltypes.otypes
FROM basic
JOIN alltypes ON oid = oidref
WHERE basic.otype = 'Sy1'
    OR basic.otype = 'Sy2'
    OR basic.otype = 'SyG'
""")

We save the date in a variable for use later in the file names of the results

In [3]:
today = date.today()

The sample results is saved in a .csv file:

In [4]:
simbad_sample_Sy = result.to_table()
simbad_sample_Sy.write(today.strftime('%b_%d_%Y')+'_Sy_Samples'+'.csv',
                    format='ascii.csv',overwrite=True)
len(simbad_sample_Sy)

43708

Using **SkyCoord** from astropy we extract the coordinates information for each object from the SIMBAD samples. This is because when we search the information in [NED](https://ned.ipac.caltech.edu/) using the object id that we obtained from SIMBAD, sometimes it doesn't reconigze it, so we will use coordinates instead

In [5]:
cat_smb_sample = SkyCoord(ra=simbad_sample_Sy['ra'], dec=simbad_sample_Sy['dec'])
cat_smb_sample[2000]

<SkyCoord (ICRS): (ra, dec) in deg
    (4.3025995, 6.30660291)>

The wavelenght for NeII, NeIII and NeV in the infrared spectrum are 12.8 microns, 15.6 microns and 14.3 microns respectively. This values are equivalents to the following frequencies which ara related to different labels of "Observed Passband" in NED database as we ilustrate in the next list:

* Frequency: 1.92e+13 Hz: [NeIII] 15.6 microns
* Frecuency: 1.93e+13 Hz: [Ne III] 15.56 (IRS), [NeIII] 15.6 Spitzer, [Ne III] (Spitzer), [Ne III] 15.5 (IRS)
* Frecuency: 2.09e+13 Hz: [Ne V] 14.32 (IRS), [Ne V] 14.32 Spitzer, [Ne V] 14.32, [Ne V] 14.3 Spitzer, [Ne V] 14.3 microns
* Frequency: 2.10e+13 Hz: [Ne V] 14.3 (IRS)
* Frequency: 2.34e+13 Hz: [Ne II] 12.81 (IRS), [Ne II] 12.8 Spitzer, [Ne II] 12.8 (IRS), [Ne II] 12.81, NEII (VLT), 12.81 microns (VLT) 

In the variable **Ne_IR_Freq** we define a list with the those frequency values

In [6]:
Ne_IR_Freq = [19200000000000.0,19300000000000.0,20900000000000.0,21000000000000.0,23400000000000.0]

Using the coordinates extracted with SkyCoord and using [Astroquery](https://astroquery.readthedocs.io/en/latest/index.html) package we will get the information from NED for each object in SIMBAD sample. Then we took the **Object Name** to get the **Photometry table** of this objects. The filter for the objects with Ne Line Emissions in Infrared will be looking in the column **"Frequency"** of the table the values of the list defined in Ne_IR_Freq.

In [12]:
total_obj_Ne = 0 #Variable for counting the objects number with data in Ne-IR
Ne_IR_obj = [] #List of objects with Ne-IR data

outfile = open('Obj_with_Ne-IR.txt','w') #We create and open a file to write the list of objects names with Ne_IR data
headerstring = 'Objects with Ne II, Ne III and Ne V data:\n' #Creating the header
outfile.write(headerstring) #Writing the header

start = time.time()
for i in range(6000):
    
    try:
        result_table = Ned.query_region(cat_smb_sample[i],radius=0.0005 * u.deg) #For obtaining the data from NED for each sample
        #print(result_table) # an astropy.table.Table
        obj_name=result_table["Object Name"] #Preferred object name in NED
    except:
        pass
    
    k=i #Flag to print only the photometric data of objects with Ne-IR data
    
    try:  
        phot_table = Ned.get_table(obj_name, table='photometry') #We get the photometry table from NED for each sample
        len_table = len(phot_table) #Lenght of the photometry data

        for j in range(len_table):
            Freq=phot_table["Frequency"][j] #We get the values of the Frequencies observed
            pt=phot_table["Observed Passband","Frequency","Flux Density"][j]

            if (Freq==Ne_IR_Freq[0] or Freq==Ne_IR_Freq[1] or Freq==Ne_IR_Freq[2] or Freq==Ne_IR_Freq[3] or 
                Freq==Ne_IR_Freq[4]): 
                
                if k==i:
                    total_obj_Ne+=1 #Increase the counter
                    print('\n')
                    print(i)
                    print(obj_name)
                    #Writing the object names
                    outstring = f'\n{obj_name[0]}'
                    outfile.write(outstring) 
                    Ne_IR_obj.append(obj_name[0]) #Save the object name in the list
                    print(phot_table["Observed Passband","Frequency","Flux Density"][j]) 
                else:
                    print(pt[0],'',pt[1],'',pt[2])
            
                k+=1  
    
    except:
        pass 

end = time.time()
#we close the file with objects names
outfile.close()

print('\nTotal objects with Ne II or Ne III or Ne V data found: ',total_obj_Ne)
print('Execution Time: ' + str(end - start))   



0
       Object Name       
-------------------------
WISEA J140931.24+051131.6
 Observed Passband     Frequency     Flux Density
                           Hz             Jy     
------------------- ---------------- ------------
[Ne II] 12.81 (IRS) 23400000000000.0 1330000000.0
[Ne V] 14.32 (IRS)  20900000000000.0  --
[Cl II] 14.37 (IRS)  20900000000000.0  --
[Ne III] 15.56 (IRS)  19300000000000.0  2010000000.0


23
 Object Name  
--------------
ESO 362- G 018
Observed Passband     Frequency      Flux Density
                          Hz              Jy     
------------------ ---------------- -------------
[Ne II] 12.8 (IRS) 23400000000000.0 16000000000.0
[Ne II] 12.81 (IRS)  23400000000000.0  9940000000.0
[Ne II] 12.81  23400000000000.0  11000000000.0
[Ne II] 12.81 (IRS)  23400000000000.0  12400000000.0
[Ne II] (Spitzer)  23400000000000.0  11000000000.0
[Ne V] 14.32 (IRS)  20900000000000.0  3270000000.0
[Ne V] 14.32 (IRS)  20900000000000.0  3150000000.0
[Ne III] 15.56  19300000000



2877
Object Name
-----------
PG 0052+251
Observed Passband     Frequency     Flux Density
                          Hz             Jy     
------------------ ---------------- ------------
[Ne II] 12.8 (IRS) 23400000000000.0 2100000000.0


2878
Object Name
-----------
   MRK 0478
Observed Passband    Frequency     Flux Density
                         Hz             Jy     
----------------- ---------------- ------------
    [Ne II] (IRS) 23400000000000.0 4110000000.0
[Ne II] 12.8 (IRS)  23400000000000.0  4520000000.0
[Ne II] 12.8 Spitzer  23400000000000.0  5900000000.0
[Ne II] 12.81 (IRS)  23400000000000.0  4270000000.0
[Ne V] (IRS)  21000000000000.0  1330000000.0
[Ne V] 14.3 (IRS)  20900000000000.0  1830000000.0
[Ne V] 14.32 (IRS)  20900000000000.0  1430000000.0
[Ne III] 15 Spitzer  19300000000000.0  5200000000.0
[Ne III] 15.56 (IRS)  19300000000000.0  3860000000.0
[Ne III] 15.5 (IRS)  19300000000000.0  3740000000.0


2993
Object Name
-----------
  UGC 08823
 Observed Passband     F



4711
 Object Name  
--------------
ESO 141- G 055
Observed Passband    Frequency     Flux Density
                         Hz             Jy     
----------------- ---------------- ------------
    [Ne II] 12.81 23400000000000.0 2240000000.0
[Ne V] 14.32  20900000000000.0  2250000000.0
[Ne III] 15.56  19300000000000.0  5620000000.0


4723
Object Name 
------------
SBS 1133+572
 Observed Passband     Frequency      Flux Density
                           Hz              Jy     
------------------- ---------------- -------------
[Ne II] 12.81 (IRS) 23400000000000.0 10800000000.0
[Ne V] 14.32 (IRS)  20900000000000.0  6990000000.0
[Ne III] 15.56 (IRS)  19300000000000.0  10500000000.0


4726
Object Name
-----------
   MRK 0509
Observed Passband     Frequency      Flux Density
                          Hz              Jy     
------------------ ---------------- -------------
[Ne II] 12.8 (IRS) 23400000000000.0 16000000000.0
[Ne II] 12.81 (IRS)  23400000000000.0  11800000000.0
[Ne II] 12.81



5452
Object Name
-----------
PG 0026+129
Observed Passband    Frequency     Flux Density
                         Hz             Jy     
----------------- ---------------- ------------
    [Ne II] (IRS) 23400000000000.0  229000000.0
[Ne II] 12.81 (IRS)  23400000000000.0  256000000.0
[Ne V] (IRS)  21000000000000.0  473000000.0
[Ne V] 14.32 Spitzer  20900000000000.0  380000000.0
[Ne V] 14.32 (IRS)  20900000000000.0  --
[Ne III] 15.56 (IRS)  19300000000000.0  --


5453
Object Name
-----------
PG 0844+349
 Observed Passband      Frequency     Flux Density
                            Hz             Jy     
-------------------- ---------------- ------------
[Ne V] 14.32 Spitzer 20900000000000.0           --


5454
Object Name
-----------
   NGC 4748
Observed Passband     Frequency      Flux Density
                          Hz              Jy     
------------------ ---------------- -------------
[Ne II] 12.8 (IRS) 23400000000000.0 12000000000.0
[Ne II] (Spitzer)  23400000000000.0  1400000



5921
Object Name
-----------
   MRK 1383
 Observed Passband     Frequency     Flux Density
                           Hz             Jy     
------------------- ---------------- ------------
[Ne II] 12.81 (IRS) 23400000000000.0 1350000000.0
[Ne II] 12.8 (IRS)  23400000000000.0  1230000000.0
[Ne II] (IRS)  23400000000000.0  1290000000.0
[Ne V] (IRS)  21000000000000.0  1250000000.0
[Ne V] 14.32 (IRS)  20900000000000.0  847000000.0
[Ne V] 14.3 (IRS)  20900000000000.0  1440000000.0
[Ne V] 14.32 Spitzer  20900000000000.0  1200000000.0
[Ne III] 15.5 (IRS)  19300000000000.0  2630000000.0
[Ne III] 15.56 (IRS)  19300000000000.0  2560000000.0

Total objects with Ne II or Ne III or Ne V data found:  70
Execution Time: 5247.495946884155


In [14]:
Ne_IR_obj

['WISEA J140931.24+051131.6',
 'ESO 362- G 018',
 '3C 334',
 'ESO 323- G 077',
 'NGC 1241',
 'PGC1 0033623 GROUP',
 'WISEA J152238.08+333136.0',
 'MRK 0937',
 'ESO 297- G 018',
 'IC 0450',
 'UGC 04229',
 'WISEA J035734.06-053719.6',
 'WISEA J210358.75-072802.6',
 'WISEA J155041.49-035318.0',
 'UGC 09214',
 'LBQS 0008-0001',
 'WISEA J005439.78+155447.2',
 'NGC 4992',
 'WISEA J141120.48+521210.6',
 'PG 0052+251',
 'MRK 0478',
 'UGC 08823',
 '[HB89] 0050+124 ABS01',
 'PG 1211+143',
 'SSTSL2 J054236.09+495107.3',
 '3C 098',
 'FAIRALL 0009',
 'NGC 2110',
 'UGC 03351',
 'NGC 4235',
 'NGC 6860',
 'MCG +04-48-002',
 '3C 309.1',
 'CGCG 011-076',
 'CGCG 218-007',
 'SBS 0934+546',
 'WISEA J101756.90+390528.0',
 'ESO 141- G 055',
 'SBS 1133+572',
 'MRK 0509',
 'NGC 7591',
 'UGC 10120',
 'MRK 1239',
 'SDSS J125614.10+565225.3',
 'WISEA J133624.05+391731.1',
 'WISEA J165939.77+183436.7',
 'GALEXASC J161357.09+654311.8',
 'VII Zw 244',
 'NGC 3982',
 'NGC 5728',
 'WISEA J005356.23-703804.2',
 'PG 0026

In [15]:
#Celda para pruebas

result_t = Ned.query_region(cat_smb_sample[5921],radius=0.0005 * u.deg) #For obtaining the data from NED for each sample 

o_name=result_t["Object Name"]
try:
  phot_t = Ned.get_table(o_name, table='photometry')
  #print(phot_t['Frequency'])
  len_t=len(phot_t)
  for a in range(len_t):
    Passb=phot_t['Frequency'][a] #We get the name for each Observed Passband
    if (Passb==Ne_IR_Freq[0] or Passb==Ne_IR_Freq[1] or Passb==Ne_IR_Freq[2] or Passb==Ne_IR_Freq[3] or Passb==Ne_IR_Freq[4]):
      print(o_name[0])
except:
  pass

MRK 1383
MRK 1383
MRK 1383
MRK 1383
MRK 1383
MRK 1383
MRK 1383
MRK 1383
MRK 1383


In [17]:
%load_ext watermark
%watermark -a "Jonhatan Bernal" -d -v -m
print('Specific Python packages')
%watermark -iv -w --packages astropy

Author: Jonhatan Bernal

Python implementation: CPython
Python version       : 3.8.8
IPython version      : 7.26.0

Compiler    : MSC v.1928 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 142 Stepping 12, GenuineIntel
CPU cores   : 8
Architecture: 64bit

Specific Python packages
astropy: 4.3.1

numpy  : 1.20.3
pyvo   : 1.1
astropy: 4.3.1

Watermark: 2.2.0

